In [16]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

import cv2
import skimage
from skimage.feature import hog

from sklearn.model_selection import train_test_split

import os
import gzip
import pickle
from tqdm.auto import tqdm
from ast import literal_eval
from sklearn.neural_network import MLPClassifier



pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [17]:
images = np.load("image.npy")
image_labels = np.load("image_category.npy")

In [18]:
np.unique(image_labels)

array([ 1, 17, 18, 27, 64])

In [133]:
def extract_HOG_features(data):
    num_samples = data.shape[0]
    hog_features = []
    for i in tqdm(range(num_samples)):
        img = data[i]
        feature = hog(img, pixels_per_cell=(8, 8), cells_per_block=(3, 3))
        hog_features.append(feature)
        if i %20 == 0:
            print()
    return np.array(hog_features)

In [134]:
images[0]

array([[0.56171918, 0.55833954, 0.55830842, ..., 0.65200786, 0.65119087,
        0.65496852],
       [0.55409445, 0.55731026, 0.55812929, ..., 0.64916878, 0.64994478,
        0.65564197],
       [0.55720017, 0.54967472, 0.554354  , ..., 0.64869231, 0.64866833,
        0.65279026],
       ...,
       [0.25084662, 0.23687481, 0.22582936, ..., 0.10068557, 0.14151743,
        0.15914492],
       [0.24338198, 0.24431225, 0.23506392, ..., 0.09660949, 0.11815671,
        0.11430642],
       [0.23657955, 0.23595347, 0.23316385, ..., 0.07665328, 0.05550884,
        0.05661618]])

In [148]:
train_features, test_features, train_labels, test_labels = train_test_split(images, image_labels)

In [149]:
train_features = extract_HOG_features(train_features)
test_features = extract_HOG_features(test_features)

  0%|          | 0/2224 [00:00<?, ?it/s]

  0%|          | 0/742 [00:00<?, ?it/s]

In [176]:
clf = MLPClassifier(solver='lbfgs', 
                    activation='relu', 
                    alpha=10, 
                    hidden_layer_sizes=(256, 64, 5), 
                    random_state=1, 
                    max_iter=500,
                    early_stopping=True,
                    verbose=True)

In [177]:
clf.fit(train_features, train_labels)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =      4081315     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.08899D+00    |proj g|=  1.74609D-01


 This problem is unconstrained.



At iterate    1    f=  3.06786D+00    |proj g|=  1.53382D-01

At iterate    2    f=  3.03207D+00    |proj g|=  1.38758D-01

At iterate    3    f=  2.97803D+00    |proj g|=  1.11431D-01

At iterate    4    f=  2.87625D+00    |proj g|=  4.81861D-02

At iterate    5    f=  2.84041D+00    |proj g|=  4.42893D-02

At iterate    6    f=  2.82314D+00    |proj g|=  5.09139D-02

At iterate    7    f=  2.79609D+00    |proj g|=  4.35614D-02

At iterate    8    f=  2.77835D+00    |proj g|=  6.84713D-02

At iterate    9    f=  2.72804D+00    |proj g|=  4.53667D-01

At iterate   10    f=  2.62036D+00    |proj g|=  2.15626D-01

At iterate   11    f=  2.53353D+00    |proj g|=  3.88445D-01

At iterate   12    f=  2.50812D+00    |proj g|=  3.29984D-01

At iterate   13    f=  2.47918D+00    |proj g|=  3.45658D-01

At iterate   14    f=  2.40709D+00    |proj g|=  3.73167D-01

At iterate   15    f=  2.35289D+00    |proj g|=  3.77235D-01

At iterate   16    f=  2.29767D+00    |proj g|=  2.97846D-01

At iter

/Users/grantwilson/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=10, early_stopping=True, hidden_layer_sizes=(256, 64, 5),
              max_iter=500, random_state=1, solver='lbfgs', verbose=True)

In [178]:
pred = clf.predict(train_features)
train_accuracy = np.mean(pred == train_labels)

pred = clf.predict(test_features)
proba = clf.predict_proba(test_features)
test_accuracy = np.mean(pred == test_labels)

In [179]:
print("Training accuracy: {}".format(train_accuracy))
print("Testing accuracy: {}".format(test_accuracy))

Training accuracy: 0.9797661870503597
Testing accuracy: 0.3463611859838275
